## Data Cleaning + Evaluating Patterns in Alphafold Protein Prediction Confidence 

In [3]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
#conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','64g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_1312/4264169132.py:8 

In [4]:
# imports
import os
from pyspark.sql.functions import col, lit, length
from  pyspark.sql.functions import input_file_name
from pyspark.sql.functions import split, substring_index, regexp_replace
from pyspark.sql.functions import collect_list
from pyspark.sql.window import Window
from pyspark.sql.functions import mean, size, udf
from pyspark.sql.types import FloatType, IntegerType, StringType
from collections import Counter
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import array
import numpy as np
from pyspark.sql.types import MapType, StringType, FloatType

os.getcwd()

'/home/jovyan/bd_final_project'

### Load in .cif files to pyspark df and perform filtering 

In [5]:
# read the input text file
input_file_paths = "/home/jovyan/bd_final_project/AF_data1/"

# read the large text files with custom line separator
df = spark.read.option("lineSep", "\n").text(input_file_paths)

# add filename
df = df.withColumn("filename", input_file_name())

# Use the `split()` function to split the "filename" column based on '/' and access the last element (the file name)
# probably need a udf for this 

# Filter lines that start with the letter 'A'
df_filtered = df.filter(df.value.startswith("A "))

# Filter rows based on the length of the line - this ensures we are only pulling amino acid and confidence scores
max_length = 25 # potential param to adjust if bugs
df_filtered = df_filtered.filter((length(df_filtered.value) <= max_length))

# Use a regex pattern to split the "value" column based on one or more whitespace characters
split_values = df_filtered.withColumn("split_value", split(df_filtered.value, "\\s+"))

# Create individual columns from the array
df_final = split_values.withColumn("comp_id", col("split_value").getItem(1)) \
                       .withColumn("metric_value", col("split_value").getItem(4)) \
                       .withColumn("ordinal_id", col("split_value").getItem(6))

# Drop the "split_value" array column since we don't need it anymore
df_final = df_final.drop("split_value")
# Drop the "value" column from the DataFrame
df_final = df_final.drop("value")


In [6]:
df_filtered.show()

+--------------------+--------------------+
|               value|            filename|
+--------------------+--------------------+
|A MET 1   2 43.25...|file:///home/jovy...|
|A SER 2   2 57.30...|file:///home/jovy...|
|A ALA 3   2 59.96...|file:///home/jovy...|
|A ARG 4   2 60.22...|file:///home/jovy...|
|A GLU 5   2 53.22...|file:///home/jovy...|
|A VAL 6   2 59.43...|file:///home/jovy...|
|A ALA 7   2 58.46...|file:///home/jovy...|
|A VAL 8   2 57.16...|file:///home/jovy...|
|A LEU 9   2 47.80...|file:///home/jovy...|
|A LEU 10  2 44.60...|file:///home/jovy...|
|A LEU 11  2 41.12...|file:///home/jovy...|
|A TRP 12  2 34.89...|file:///home/jovy...|
|A LEU 13  2 34.04...|file:///home/jovy...|
|A SER 14  2 37.35...|file:///home/jovy...|
|A CYS 15  2 34.99...|file:///home/jovy...|
|A TYR 16  2 41.30...|file:///home/jovy...|
|A GLY 17  2 46.60...|file:///home/jovy...|
|A SER 18  2 56.73...|file:///home/jovy...|
|A ALA 19  2 73.88...|file:///home/jovy...|
|A LEU 20  2 86.69...|file:///ho

In [7]:
df_final.show()

+--------------------+-------+------------+----------+
|            filename|comp_id|metric_value|ordinal_id|
+--------------------+-------+------------+----------+
|file:///home/jovy...|    MET|       43.25|         1|
|file:///home/jovy...|    SER|       57.30|         2|
|file:///home/jovy...|    ALA|       59.96|         3|
|file:///home/jovy...|    ARG|       60.22|         4|
|file:///home/jovy...|    GLU|       53.22|         5|
|file:///home/jovy...|    VAL|       59.43|         6|
|file:///home/jovy...|    ALA|       58.46|         7|
|file:///home/jovy...|    VAL|       57.16|         8|
|file:///home/jovy...|    LEU|       47.80|         9|
|file:///home/jovy...|    LEU|       44.60|        10|
|file:///home/jovy...|    LEU|       41.12|        11|
|file:///home/jovy...|    TRP|       34.89|        12|
|file:///home/jovy...|    LEU|       34.04|        13|
|file:///home/jovy...|    SER|       37.35|        14|
|file:///home/jovy...|    CYS|       34.99|        15|
|file:///h

In [8]:
df_final.dtypes

[('filename', 'string'),
 ('comp_id', 'string'),
 ('metric_value', 'string'),
 ('ordinal_id', 'string')]

Check to see if first and last proteins a are sequentially mapped in our database

In [9]:
df_final.head(10)

[Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='MET', metric_value='43.25', ordinal_id='1'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='SER', metric_value='57.30', ordinal_id='2'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='ALA', metric_value='59.96', ordinal_id='3'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='ARG', metric_value='60.22', ordinal_id='4'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='GLU', metric_value='53.22', ordinal_id='5'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='VAL', metric_value='59.43', ordinal_id='6'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q5SY80-F1-model_v4.cif', comp_id='ALA', metric_value='58.46', ordinal_id='7'),

In [10]:
df_final.tail(10)

[Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='HIS', metric_value='51.10', ordinal_id='35'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='GLY', metric_value='48.48', ordinal_id='36'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='CYS', metric_value='51.34', ordinal_id='37'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='HIS', metric_value='50.43', ordinal_id='38'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='SER', metric_value='50.87', ordinal_id='39'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='SER', metric_value='47.47', ordinal_id='40'),
 Row(filename='file:///home/jovyan/bd_final_project/AF_data1/AF-Q3LI62-F1-model_v4.cif', comp_id='TYR', metric_value='45.68', ordinal_id

Additional column processing

In [11]:
df_final = df_final.repartition(10)

# Split the 'filename' column based on '/'
df_final = df_final.withColumn("filename_split", split(col("filename"), "/"))
df_final = df_final.withColumn("protein_id", col("filename_split").getItem(7))

# Drop the "split_value" array column since we don't need it anymore
df_final = df_final.drop("filename_split")
# Drop the "value" column from the DataFrame
df_final = df_final.drop("filename")

# Clean the protein_id column to remove 'AF-' at the beginning and '-F1-model...' at the end
df_final = df_final.withColumn("protein_id", regexp_replace(col("protein_id"), "^AF-|-F1-model.*$", ""))

# Reorder columns to place 'protein_id' as the first column
df_final = df_final.select("protein_id", "comp_id", "metric_value", "ordinal_id")

# Show the final DataFrame
df_final.show(truncate=True)

+----------+-------+------------+----------+
|protein_id|comp_id|metric_value|ordinal_id|
+----------+-------+------------+----------+
|    P15144|    MET|       95.50|       212|
|    Q2TAC6|    SER|       95.41|       324|
|    Q5SY80|    TYR|       89.82|       536|
|    P15144|    PHE|       98.54|       221|
|    Q0P6D6|    VAL|       95.65|        91|
|    P33176|    ASN|       81.56|       501|
|    Q2TAC6|    ILE|       52.01|       650|
|    P15144|    SER|       94.19|       307|
|    P15144|    ASP|       94.21|       370|
|    Q2TAC6|    GLU|       28.73|       704|
|    Q2KHM9|    THR|       30.71|       135|
|    Q2KHM9|    ALA|       91.50|       369|
|    Q0P6D6|    LYS|       92.40|       105|
|    P57679|    THR|       91.18|       542|
|    P57679|    LEU|       96.07|       216|
|    P15144|    VAL|       86.48|       822|
|    Q2TAC6|    LEU|       38.52|       409|
|    P54760|    ALA|       89.27|       371|
|    P54760|    VAL|       98.36|       809|
|    P5476

Collect amino acids and confidence scores on a protein-wise basis and transpose them into vectors. 

In [12]:
# Repartition the DataFrame based on 'protein_id' and sort within each partition
#df_repartitioned = df_final.repartition("protein_id").sortWithinPartitions("protein_id", "ordinal_id")

# Group by 'protein_id' and aggregate 'comp_id', 'metric_value', and 'ordinal_id' into lists
df_grouped = df_final.groupBy("protein_id").agg(
    collect_list("comp_id").alias("comp_id_list"),
    collect_list("metric_value").alias("metric_value_list"),
    collect_list("ordinal_id").alias("ordinal_id_list")
)

df_grouped.show()

+----------+--------------------+--------------------+--------------------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|
+----------+--------------------+--------------------+--------------------+
|    P01764|[SER, GLY, GLY, A...|[96.44, 96.30, 97...|[44, 28, 27, 116,...|
|    P01833|[THR, CYS, VAL, T...|[33.33, 93.66, 40...|[722, 441, 672, 3...|
|    P02545|[TYR, PHE, TYR, A...|[35.80, 96.15, 96...|[626, 451, 81, 47...|
|    P02656|[VAL, THR, VAL, A...|[86.55, 83.18, 62...|[8, 76, 55, 56, 1...|
|    P02679|[ASP, LYS, ILE, A...|[98.37, 93.10, 98...|[314, 114, 182, 1...|
|    P02765|[VAL, LYS, GLU, V...|[96.92, 93.04, 39...|[110, 106, 262, 2...|
|    P02771|[LYS, GLN, SER, H...|[94.77, 92.10, 91...|[68, 394, 115, 12...|
|    P03886|[SER, PHE, ALA, L...|[96.12, 89.71, 85...|[306, 220, 249, 2...|
|    P03891|[SER, THR, LEU, L...|[91.17, 97.82, 97...|[148, 185, 130, 2...|
|    P03951|[GLU, TYR, PHE, G...|[42.08, 29.43, 89...|[379, 5, 295, 453...|
|    P04066|

Define a custom function to reorder comp_id_list and metric_value_list based on the order of ordinal_id_list

In [13]:
def reorder_lists(comp_id_list, metric_value_list, ordinal_id_list):
    # Create a list of tuples with comp_id, metric_value, and ordinal_id
    combined_list = list(zip(comp_id_list, metric_value_list, ordinal_id_list))
    
    # Sort the combined list based on ordinal_id
    combined_list.sort(key=lambda x: int(x[2]))
    
    # Unzip the sorted list back into separate lists
    reordered_comp_id_list, reordered_metric_value_list, reordered_ordinal_id_list = zip(*combined_list)
    
    return list(reordered_comp_id_list), list(reordered_metric_value_list), list(reordered_ordinal_id_list)


In [14]:
# Convert the function to a UDF
reorder_lists_udf = udf(reorder_lists, ArrayType(StringType()))

# Apply the UDF to reorder comp_id_list and metric_value_list based on ordinal_id_list
df_reordered = df_grouped.withColumn("reordered_lists", reorder_lists_udf("comp_id_list", "metric_value_list", "ordinal_id_list"))


# Split the reordered_lists column into reordered_comp_id_list and reordered_metric_value_list
df_final = df_reordered.withColumn("comp_id_list", col("reordered_lists")[0]) \
                       .withColumn("metric_value_list", col("reordered_lists")[1]) \
                       .withColumn("ordinal_id_list", col("reordered_lists")[2]) \
                       .drop("reordered_lists")

# Show the final DataFrame
df_final.show()

+----------+--------------------+--------------------+--------------------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|
+----------+--------------------+--------------------+--------------------+
|    P01764|[MET, GLU, PHE, G...|[41.82, 45.26, 54...|[1, 2, 3, 4, 5, 6...|
|    P01833|[MET, LEU, LEU, P...|[52.94, 50.97, 50...|[1, 2, 3, 4, 5, 6...|
|    P02545|[MET, GLU, THR, P...|[35.94, 32.07, 36...|[1, 2, 3, 4, 5, 6...|
|    P02656|[MET, GLN, PRO, A...|[53.90, 69.83, 77...|[1, 2, 3, 4, 5, 6...|
|    P02679|[MET, SER, TRP, S...|[36.93, 34.45, 38...|[1, 2, 3, 4, 5, 6...|
|    P02765|[MET, LYS, SER, L...|[59.39, 65.13, 70...|[1, 2, 3, 4, 5, 6...|
|    P02771|[MET, LYS, TRP, V...|[33.35, 34.98, 34...|[1, 2, 3, 4, 5, 6...|
|    P03886|[MET, PRO, MET, A...|[67.59, 82.83, 90...|[1, 2, 3, 4, 5, 6...|
|    P03891|[MET, ASN, PRO, L...|[81.27, 90.75, 92...|[1, 2, 3, 4, 5, 6...|
|    P03951|[MET, ILE, PHE, L...|[35.03, 31.60, 30...|[1, 2, 3, 4, 5, 6...|
|    P04066|

Convert back to array dtypes for downstream processing. 

In [15]:
df_final.dtypes

[('protein_id', 'string'),
 ('comp_id_list', 'string'),
 ('metric_value_list', 'string'),
 ('ordinal_id_list', 'string')]

In [16]:
df_final.printSchema()


root
 |-- protein_id: string (nullable = true)
 |-- comp_id_list: string (nullable = true)
 |-- metric_value_list: string (nullable = true)
 |-- ordinal_id_list: string (nullable = true)



In [17]:
# Remove leading and trailing brackets and convert to appropriate array types
df_final = df_final \
    .withColumn("comp_id_list", split(regexp_replace(col("comp_id_list"), r"[\[\]]", ""), ",").cast(ArrayType(StringType()))) \
    .withColumn("metric_value_list", split(regexp_replace(col("metric_value_list"), r"[\[\]]", ""), ",").cast(ArrayType(FloatType()))) \
    .withColumn("ordinal_id_list", split(regexp_replace(col("ordinal_id_list"), r"[\[\]]", ""), ",").cast(ArrayType(IntegerType())))

# Show the adjusted DataFrame
df_final.show()

+----------+--------------------+--------------------+--------------------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|
+----------+--------------------+--------------------+--------------------+
|    P01764|[MET,  GLU,  PHE,...|[41.82, 45.26, 54...|[1, 2, 3, 4, 5, 6...|
|    P01833|[MET,  LEU,  LEU,...|[52.94, 50.97, 50...|[1, 2, 3, 4, 5, 6...|
|    P02545|[MET,  GLU,  THR,...|[35.94, 32.07, 36...|[1, 2, 3, 4, 5, 6...|
|    P02656|[MET,  GLN,  PRO,...|[53.9, 69.83, 77....|[1, 2, 3, 4, 5, 6...|
|    P02679|[MET,  SER,  TRP,...|[36.93, 34.45, 38...|[1, 2, 3, 4, 5, 6...|
|    P02765|[MET,  LYS,  SER,...|[59.39, 65.13, 70...|[1, 2, 3, 4, 5, 6...|
|    P02771|[MET,  LYS,  TRP,...|[33.35, 34.98, 34...|[1, 2, 3, 4, 5, 6...|
|    P03886|[MET,  PRO,  MET,...|[67.59, 82.83, 90...|[1, 2, 3, 4, 5, 6...|
|    P03891|[MET,  ASN,  PRO,...|[81.27, 90.75, 92...|[1, 2, 3, 4, 5, 6...|
|    P03951|[MET,  ILE,  PHE,...|[35.03, 31.6, 30....|[1, 2, 3, 4, 5, 6...|
|    P04066|

In [18]:
df_final.printSchema()


root
 |-- protein_id: string (nullable = true)
 |-- comp_id_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- metric_value_list: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ordinal_id_list: array (nullable = true)
 |    |-- element: integer (containsNull = true)



Compute some basic descriptive stats

In [19]:
from pyspark.sql.functions import size

# Compute the length of the 'ordinal_id_list' column and create a new column called 'protein length'
df_final = df_final.withColumn("protein_seq_len", size(col("ordinal_id_list")))

# Show the final DataFrame with the new column
df_final.show()

+----------+--------------------+--------------------+--------------------+---------------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|protein_seq_len|
+----------+--------------------+--------------------+--------------------+---------------+
|    P01764|[MET,  GLU,  PHE,...|[41.82, 45.26, 54...|[1, 2, 3, 4, 5, 6...|            117|
|    P01833|[MET,  LEU,  LEU,...|[52.94, 50.97, 50...|[1, 2, 3, 4, 5, 6...|            764|
|    P02545|[MET,  GLU,  THR,...|[35.94, 32.07, 36...|[1, 2, 3, 4, 5, 6...|            664|
|    P02656|[MET,  GLN,  PRO,...|[53.9, 69.83, 77....|[1, 2, 3, 4, 5, 6...|             99|
|    P02679|[MET,  SER,  TRP,...|[36.93, 34.45, 38...|[1, 2, 3, 4, 5, 6...|            453|
|    P02765|[MET,  LYS,  SER,...|[59.39, 65.13, 70...|[1, 2, 3, 4, 5, 6...|            367|
|    P02771|[MET,  LYS,  TRP,...|[33.35, 34.98, 34...|[1, 2, 3, 4, 5, 6...|            609|
|    P03886|[MET,  PRO,  MET,...|[67.59, 82.83, 90...|[1, 2, 3, 4, 5, 6...|     

In [20]:
#e a UDF to compute the mean of the metric_value_list
def compute_mean(values):
    return float(np.mean(values))

# Register the UDF with a return type of FloatType
mean_metric_udf = udf(compute_mean, FloatType())

# Add a new column called 'mean_metric' to the DataFrame using the UDF
df_final = df_final.withColumn("mean_metric", mean_metric_udf(col("metric_value_list")))

# Show the final DataFrame with the new column
df_final.show()

+----------+--------------------+--------------------+--------------------+---------------+-----------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|protein_seq_len|mean_metric|
+----------+--------------------+--------------------+--------------------+---------------+-----------+
|    P01764|[MET,  GLU,  PHE,...|[41.82, 45.26, 54...|[1, 2, 3, 4, 5, 6...|            117|   91.02701|
|    P01833|[MET,  LEU,  LEU,...|[52.94, 50.97, 50...|[1, 2, 3, 4, 5, 6...|            764|   76.67788|
|    P02545|[MET,  GLU,  THR,...|[35.94, 32.07, 36...|[1, 2, 3, 4, 5, 6...|            664|  76.273735|
|    P02656|[MET,  GLN,  PRO,...|[53.9, 69.83, 77....|[1, 2, 3, 4, 5, 6...|             99|   68.22879|
|    P02679|[MET,  SER,  TRP,...|[36.93, 34.45, 38...|[1, 2, 3, 4, 5, 6...|            453|   85.61137|
|    P02765|[MET,  LYS,  SER,...|[59.39, 65.13, 70...|[1, 2, 3, 4, 5, 6...|            367|  76.515884|
|    P02771|[MET,  LYS,  TRP,...|[33.35, 34.98, 34...|[1, 2, 3, 

In [21]:
from collections import Counter

# Define a UDF to compute the most common value in the comp_id_list
def most_common_value(comp_id_list):
    # Use Counter to calculate frequencies of each value in the list
    frequency = Counter(comp_id_list)
    # Find the most common value and return it
    most_common_aa = frequency.most_common(1)[0][0]
    return most_common_aa

# Register the UDF with a return type of StringType
most_common_aa_udf = udf(most_common_value, StringType())

# Add a new column called 'most_common_aa' to the DataFrame using the UDF
df_final = df_final.withColumn("most_common_aa", most_common_aa_udf(col("comp_id_list")))

# Show the final DataFrame with the new column
df_final.show()

+----------+--------------------+--------------------+--------------------+---------------+-----------+--------------+
|protein_id|        comp_id_list|   metric_value_list|     ordinal_id_list|protein_seq_len|mean_metric|most_common_aa|
+----------+--------------------+--------------------+--------------------+---------------+-----------+--------------+
|    P01764|[MET,  GLU,  PHE,...|[41.82, 45.26, 54...|[1, 2, 3, 4, 5, 6...|            117|   91.02701|           SER|
|    P01833|[MET,  LEU,  LEU,...|[52.94, 50.97, 50...|[1, 2, 3, 4, 5, 6...|            764|   76.67788|           SER|
|    P02545|[MET,  GLU,  THR,...|[35.94, 32.07, 36...|[1, 2, 3, 4, 5, 6...|            664|  76.273735|           LEU|
|    P02656|[MET,  GLN,  PRO,...|[53.9, 69.83, 77....|[1, 2, 3, 4, 5, 6...|             99|   68.22879|           ALA|
|    P02679|[MET,  SER,  TRP,...|[36.93, 34.45, 38...|[1, 2, 3, 4, 5, 6...|            453|   85.61137|           LEU|
|    P02765|[MET,  LYS,  SER,...|[59.39, 65.13, 

Clean df for further processing

In [22]:
# Rename columns
df_final = df_final \
    .withColumnRenamed("comp_id_list", "protein_seq") \
    .withColumnRenamed("metric_value_list", "aa_pLDDT_list") \
    .withColumnRenamed("mean_metric", "avg_pLDDT")

# Drop the 'ordinal_id_list' column
df_final = df_final.drop("ordinal_id_list")

# Reorder columns to the specified order
df_final = df_final.select(
    "protein_id",
    "protein_seq",
    "protein_seq_len",
    "most_common_aa",
    "aa_pLDDT_list",
    "avg_pLDDT"
)

# Show the final DataFrame
df_final.show()

+----------+--------------------+---------------+--------------+--------------------+---------+
|protein_id|         protein_seq|protein_seq_len|most_common_aa|       aa_pLDDT_list|avg_pLDDT|
+----------+--------------------+---------------+--------------+--------------------+---------+
|    P01764|[MET,  GLU,  PHE,...|            117|           SER|[41.82, 45.26, 54...| 91.02701|
|    P01833|[MET,  LEU,  LEU,...|            764|           SER|[52.94, 50.97, 50...| 76.67788|
|    P02545|[MET,  GLU,  THR,...|            664|           LEU|[35.94, 32.07, 36...|76.273735|
|    P02656|[MET,  GLN,  PRO,...|             99|           ALA|[53.9, 69.83, 77....| 68.22879|
|    P02679|[MET,  SER,  TRP,...|            453|           LEU|[36.93, 34.45, 38...| 85.61137|
|    P02765|[MET,  LYS,  SER,...|            367|           PRO|[59.39, 65.13, 70...|76.515884|
|    P02771|[MET,  LYS,  TRP,...|            609|           LEU|[33.35, 34.98, 34...| 89.07504|
|    P03886|[MET,  PRO,  MET,...|       

In [23]:
from pyspark.sql.types import ArrayType, StringType

# Define a UDF to remove leading spaces from each amino acid in the protein_seq array
def remove_spaces(arr):
    return [element.strip() for element in arr]

# Register the UDF with a return type of ArrayType(StringType())
remove_spaces_udf = udf(remove_spaces, ArrayType(StringType()))

# Apply the UDF to the protein_seq column to remove leading spaces from each element
df_cleaned = df_final.withColumn("protein_seq", remove_spaces_udf(col("protein_seq")))

# Show the cleaned DataFrame
df_cleaned.show()

+----------+--------------------+---------------+--------------+--------------------+---------+
|protein_id|         protein_seq|protein_seq_len|most_common_aa|       aa_pLDDT_list|avg_pLDDT|
+----------+--------------------+---------------+--------------+--------------------+---------+
|    P01764|[MET, GLU, PHE, G...|            117|           SER|[41.82, 45.26, 54...| 91.02701|
|    P01833|[MET, LEU, LEU, P...|            764|           SER|[52.94, 50.97, 50...| 76.67788|
|    P02545|[MET, GLU, THR, P...|            664|           LEU|[35.94, 32.07, 36...|76.273735|
|    P02656|[MET, GLN, PRO, A...|             99|           ALA|[53.9, 69.83, 77....| 68.22879|
|    P02679|[MET, SER, TRP, S...|            453|           LEU|[36.93, 34.45, 38...| 85.61137|
|    P02765|[MET, LYS, SER, L...|            367|           PRO|[59.39, 65.13, 70...|76.515884|
|    P02771|[MET, LYS, TRP, V...|            609|           LEU|[33.35, 34.98, 34...| 89.07504|
|    P03886|[MET, PRO, MET, A...|       

Compute ratios

In [24]:

# Define a list of all unique amino acids (considering standard amino acids)
amino_acids = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS",
               "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]

# Define a UDF to compute the ratio of each amino acid in the protein_seq column
def calculate_aa_ratios(protein_seq):
    
    # Initialize a dictionary with each amino acid and a count of 0
    aa_counts = {aa: 0 for aa in amino_acids}
    # Count occurrences of each amino acid in the protein_seq
    for aa in protein_seq:
        if aa in aa_counts:
            aa_counts[aa] += 1
    # Calculate the total length of the protein sequence
    total_length = len(protein_seq)
    # Calculate the ratio of each amino acid
    aa_ratios = {aa: aa_counts[aa] / total_length if total_length > 0 else 0 for aa in amino_acids}
    return aa_ratios

# Register the UDF with a return type of MapType(StringType(), FloatType())
calculate_aa_ratios_udf = udf(calculate_aa_ratios, MapType(StringType(), FloatType()))

# Add a new column 'aa_ratios' to the DataFrame using the UDF
df_with_ratios = df_cleaned.withColumn("aa_ratios", calculate_aa_ratios_udf(col("protein_seq")))
df_with_ratios.head(5)

[Row(protein_id='P01764', protein_seq=['MET', 'GLU', 'PHE', 'GLY', 'LEU', 'SER', 'TRP', 'LEU', 'PHE', 'LEU', 'VAL', 'ALA', 'ILE', 'LEU', 'LYS', 'GLY', 'VAL', 'GLN', 'CYS', 'GLU', 'VAL', 'GLN', 'LEU', 'VAL', 'GLU', 'SER', 'GLY', 'GLY', 'GLY', 'LEU', 'VAL', 'GLN', 'PRO', 'GLY', 'GLY', 'SER', 'LEU', 'ARG', 'LEU', 'SER', 'CYS', 'ALA', 'ALA', 'SER', 'GLY', 'PHE', 'THR', 'PHE', 'SER', 'SER', 'TYR', 'ALA', 'MET', 'SER', 'TRP', 'VAL', 'ARG', 'GLN', 'ALA', 'PRO', 'GLY', 'LYS', 'GLY', 'LEU', 'GLU', 'TRP', 'VAL', 'SER', 'ALA', 'ILE', 'SER', 'GLY', 'SER', 'GLY', 'GLY', 'SER', 'THR', 'TYR', 'TYR', 'ALA', 'ASP', 'SER', 'VAL', 'LYS', 'GLY', 'ARG', 'PHE', 'THR', 'ILE', 'SER', 'ARG', 'ASP', 'ASN', 'SER', 'LYS', 'ASN', 'THR', 'LEU', 'TYR', 'LEU', 'GLN', 'MET', 'ASN', 'SER', 'LEU', 'ARG', 'ALA', 'GLU', 'ASP', 'THR', 'ALA', 'VAL', 'TYR', 'TYR', 'CYS', 'ALA', 'LYS'], protein_seq_len=117, most_common_aa=' SER', aa_pLDDT_list=[41.81999969482422, 45.2599983215332, 54.709999084472656, 56.939998626708984, 57.04

In [25]:
# Split the 'aa_ratios' map column into separate columns for each amino acid
for aa in amino_acids:
    df_with_ratios = df_with_ratios.withColumn(aa + "_ratio", col("aa_ratios")[aa])

# Drop the intermediate 'aa_ratios' column
df_with_ratios = df_with_ratios.drop("aa_ratios")

# Show the final DataFrame
df_with_ratios.show()

24/05/04 22:06:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------------+---------------+--------------+--------------------+---------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+
|protein_id|         protein_seq|protein_seq_len|most_common_aa|       aa_pLDDT_list|avg_pLDDT|  ALA_ratio|  ARG_ratio|  ASN_ratio|  ASP_ratio|   CYS_ratio|  GLN_ratio|  GLU_ratio|  GLY_ratio|  HIS_ratio|  ILE_ratio|  LEU_ratio|  LYS_ratio|   MET_ratio|  PHE_ratio|  PRO_ratio|  SER_ratio|  THR_ratio|   TRP_ratio|  TYR_ratio|  VAL_ratio|
+----------+--------------------+---------------+--------------+--------------------+---------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------------+--------

In [26]:
alpha_df = df_with_ratios.toPandas()

In [27]:
alpha_df.head()

,protein_id,protein_seq,protein_seq_len,most_common_aa,aa_pLDDT_list,avg_pLDDT,ALA_ratio,ARG_ratio,ASN_ratio,ASP_ratio,...,LEU_ratio,LYS_ratio,MET_ratio,PHE_ratio,PRO_ratio,SER_ratio,THR_ratio,TRP_ratio,TYR_ratio,VAL_ratio
0,P01764,"[MET, GLU, PHE, GLY, LEU, SER, TRP, LEU, PHE, ...",117,SER,"[41.81999969482422, 45.2599983215332, 54.70999...",91.027008,0.085470,0.042735,0.025641,0.025641,...,0.102564,0.042735,0.025641,0.042735,0.017094,0.136752,0.042735,0.025641,0.051282,0.076923
1,P01833,"[MET, LEU, LEU, PHE, VAL, LEU, THR, CYS, LEU, ...",764,SER,"[52.939998626708984, 50.970001220703125, 50.18...",76.677879,0.075916,0.053665,0.051047,0.049738,...,0.086387,0.054974,0.005236,0.032723,0.041885,0.089005,0.057592,0.014398,0.032723,0.085079
2,P02545,"[MET, GLU, THR, PRO, SER, GLN, ARG, ARG, ALA, ...",664,LEU,"[35.939998626708984, 32.06999969482422, 36.529...",76.273735,0.088855,0.096386,0.031627,0.054217,...,0.109940,0.060241,0.015060,0.012048,0.022590,0.103916,0.054217,0.006024,0.016566,0.046687
3,P02656,"[MET, GLN, PRO, ARG, VAL, LEU, LEU, VAL, VAL, ...",99,ALA,"[53.900001525878906, 69.83000183105469, 77.410...",68.228790,0.151515,0.040404,0.000000,0.070707,...,0.111111,0.060606,0.030303,0.040404,0.030303,0.121212,0.050505,0.030303,0.020202,0.090909
4,P02679,"[MET, SER, TRP, SER, LEU, HIS, PRO, ARG, ASN, ...",453,LEU,"[36.93000030517578, 34.45000076293945, 38.4000...",85.611366,0.061810,0.026490,0.052980,0.075055,...,0.075055,0.075055,0.019868,0.044150,0.033113,0.066225,0.066225,0.024283,0.052980,0.033113


In [28]:
alpha_df.to_csv('alpha_df.csv', index=False)

In [33]:
# Specify the columns you want to save
columns_to_save = ["protein_id", "avg_pLDDT"] + [aa + "_ratio" for aa in amino_acids]

# Select the columns from the df_with_ratios DataFrame
df_selected = df_with_ratios.select(columns_to_save)

# Convert the selected PySpark DataFrame to a Pandas DataFrame
alpha_df = df_selected.toPandas()

ERROR:root:Exception while sending command.                         (0 + 2) / 2]
Traceback (most recent call last):
  File "/opt/conda/envs/bigdata/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/bigdata/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/envs/bigdata/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/bigdata/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_co

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
alpha_df.head()

In [ ]:
alpha_df.to_csv('alpha_df.csv', index=False)  

Correlation

In [30]:
# Compute the correlation between the ratio of each amino acid and the avg_pLDDT

# Calculate the correlation matrix for the selected columns
correlation_matrix = alpha_df.corr()

# Extract the correlation of avg_pLDDT with each amino acid ratio
correlation_with_avg_pLDDT = correlation_matrix.loc["avg_pLDDT", [aa + "_ratio" for aa in amino_acids]]

# Display the correlation of avg_pLDDT with each amino acid ratio
print("Correlation between avg_pLDDT and each amino acid ratio:")
print(correlation_with_avg_pLDDT)


ValueError: could not convert string to float: 'P01764'

In [ ]:
# Compute the correlation between protein length and avg_pLDDT

# Calculate the correlation between 'protein_seq_len' and 'avg_pLDDT'
correlation_length_avg_pLDDT = alpha_df[["protein_seq_len", "avg_pLDDT"]].corr().loc["protein_seq_len", "avg_pLDDT"]

# Display the correlation
print(f"Correlation between protein length and avg_pLDDT: {correlation_length_avg_pLDDT}")


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Select the columns for the model
features_columns = ["protein_seq_len"] + [aa + "_ratio" for aa in amino_acids]
label_column = "avg_pLDDT"

# Convert the selected columns into a single feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")

# Transform the DataFrame to add the features column
df_transformed = assembler.transform(df_with_ratios)

# Split the data into training and test sets
(train_data, test_data) = df_transformed.randomSplit([0.8, 0.2], seed=42)

# Define the LinearRegression model
lr = LinearRegression(featuresCol="features", labelCol=label_column)

# Fit the model on the training data
model = lr.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model's performance using the test data
evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")

# Compute the root mean squared error (RMSE)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")